In [2]:
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer as VS

import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string
import re
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.naive_bayes import BernoulliNB


import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [4]:
data = pd.read_csv("labeled_data.csv",encodin = "latin-1")
tweets = data.tweet

In [6]:
stopwords=stopwords = nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

stemmer = PorterStemmer()
sentiment_analyzer = VS()

def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    return parsed_text

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    return tweet.split()

In [9]:
def other_features(tweet):
    sentiment = sentiment_analyzer.polarity_scores(tweet)
    features = [sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound']]
    return features

def get_feature_array(tweets):
    feats=[]
    for t in tweets:
        feats.append(other_features(t))
    return np.array(feats)

In [10]:
other_features_names = ["vader neg","vader pos","vader neu", "vader compound"]

In [11]:
tweet_tokens = []
for t in tweets:
    tokens = basic_tokenize(preprocess(t))
    tweet_tokens.append(tokens)

In [12]:
feats = get_feature_array(tweets)
M = np.concatenate([feats],axis=1)

X = pd.DataFrame(M)
y = data['class'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=69, test_size=0.33)

Logistic Regression

In [19]:
pipe = Pipeline([('select', SelectFromModel(LogisticRegression(class_weight='balanced', penalty="l1", C=0.01))),
                 ('model', LogisticRegression(class_weight='balanced',penalty='l2'))])
param_grid = [{}] 
grid_search = GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=5, random_state=69).split(X_train, y_train), verbose=2)
model = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................
[CV] ................................................. , total=   8.3s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.3s remaining:    0.0s


[CV] ................................................. , total=   7.3s
[CV]  ................................................................
[CV] ................................................. , total=   5.7s
[CV]  ................................................................
[CV] ................................................. , total=   1.6s
[CV]  ................................................................
[CV] ................................................. , total=   1.4s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   24.8s finished


In [20]:
y_preds = model.predict(X_test)
report = classification_report( y_test, y_preds )
print(report)

             precision    recall  f1-score   support

          0       0.28      0.01      0.01      8186
          1       0.53      0.65      0.59     14805
          2       0.65      0.82      0.72     16957

avg / total       0.53      0.59      0.53     39948



Random Forest

In [22]:
pipe = Pipeline([('select', SelectFromModel(LogisticRegression(class_weight='balanced',penalty="l1", C=0.01))),
                 ('model', RandomForestClassifier(n_estimators=300, random_state=0))])
param_grid = [{}]
grid_search = GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=5, random_state=69).split(X_train, y_train), verbose=2)
model = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................
[CV] ................................................. , total=  48.5s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   56.6s remaining:    0.0s


[CV] ................................................. , total=  41.3s
[CV]  ................................................................
[CV] ................................................. , total=  53.6s
[CV]  ................................................................
[CV] ................................................. , total=  52.1s
[CV]  ................................................................
[CV] ................................................. , total=  42.0s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  4.6min finished


In [23]:
y_preds = model.predict(X_test)
report = classification_report( y_test, y_preds )
print(report)

             precision    recall  f1-score   support

          0       0.64      0.47      0.54      8186
          1       0.70      0.64      0.67     14805
          2       0.68      0.82      0.74     16957

avg / total       0.68      0.68      0.67     39948



Linear SVC

In [25]:
pipe = Pipeline([('select', SelectFromModel(LogisticRegression(class_weight='balanced', penalty="l1", C=0.01))),
                 ('model', LinearSVC(C=0.05,random_state=0))])
param_grid = [{}] 
grid_search = GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=5, random_state=69).split(X_train, y_train), verbose=2)
model = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................
[CV] ................................................. , total=   7.7s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.7s remaining:    0.0s


[CV] ................................................. , total=   1.6s
[CV]  ................................................................
[CV] ................................................. , total=  11.0s
[CV]  ................................................................
[CV] ................................................. , total=   1.1s
[CV]  ................................................................
[CV] ................................................. , total=   5.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   27.0s finished


In [26]:
y_preds = model.predict(X_test)
report = classification_report( y_test, y_preds )
print(report)

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      8186
          1       0.54      0.64      0.58     14805
          2       0.63      0.84      0.72     16957

avg / total       0.47      0.59      0.52     39948



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Extra Trees

In [28]:
pipe = Pipeline([('select', SelectFromModel(LogisticRegression(class_weight='balanced', penalty="l1", C=0.01))),
                 ('model', ExtraTreeClassifier())])
# Optionally add parameters here
grid_search = GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=5, random_state=69).split(X_train, y_train), verbose=2)
model = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................
[CV] ................................................. , total=   1.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


[CV] ................................................. , total=   6.7s
[CV]  ................................................................
[CV] ................................................. , total=   1.2s
[CV]  ................................................................
[CV] ................................................. , total=   1.2s
[CV]  ................................................................
[CV] ................................................. , total=   6.1s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   16.7s finished


In [29]:
y_preds = model.predict(X_test)
report = classification_report( y_test, y_preds )
print(report)

             precision    recall  f1-score   support

          0       0.59      0.50      0.54      8186
          1       0.70      0.62      0.66     14805
          2       0.68      0.79      0.73     16957

avg / total       0.67      0.67      0.67     39948



Naive Bayes

In [31]:
pipe = Pipeline([('select', SelectFromModel(LogisticRegression(class_weight='balanced', penalty="l1", C=0.01))),
                 ('model', BernoulliNB())])
param_grid = [{}] 
grid_search = GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=5, random_state=69).split(X_train, y_train), verbose=2)
model = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................
[CV] ................................................. , total=   9.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.2s remaining:    0.0s


[CV] ................................................. , total=   1.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.7s
[CV]  ................................................................
[CV] ................................................. , total=  11.5s
[CV]  ................................................................
[CV] ................................................. , total=   7.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   30.4s finished


In [32]:
y_preds = model.predict(X_test)
report = classification_report( y_test, y_preds )
print(report)

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      8186
          1       0.50      0.75      0.60     14805
          2       0.69      0.71      0.70     16957

avg / total       0.48      0.58      0.52     39948



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
